<a href="https://colab.research.google.com/github/KhaledRobot/ML/blob/main/Court_agent_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [20]:


# Two–Lawyer Agents (Defense & Prosecution)


from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient

class LawyerAgent:


    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")           # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)

        # Ensure max_new_tokens is set if not passed
        gen_kwargs.setdefault("max_new_tokens", 300)

        completion = self.client.text_generation(
            prompt,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
    )

        answer = completion.strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer



# System prompts

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

PLAINTIFF_SYSTEM = """
You are **Garcia**, *Plaintiff* in this case. Your role:
• Clearly state what the defendant did wrong
• Explain how their actions harmed you
• Specify what compensation/remedy you seek
Style:
• Factual and concise (1-2 sentences per point)
• Avoid emotional language; focus on damages
"""

WITNESS_SYSTEM = """
You are an **eyewitness**. Rules:
• ONLY state what you directly observed
• Never speculate or guess
• If unsure, say "I don't recall"
Example responses:
• "I saw the defendant enter the server room at 11PM"
• "I don't know what they did inside"
"""

DEFENDANT_SYSTEM = """
You are the defendant. Your job:
- Deny or explain the accusations against you
- Stick to facts you personally know
- Be respectful but firm
- Admit uncertainty where needed
"""

JUDGE_SYSTEM = """
You are the **Judge**. Your role:
- Summarize briefly what each side argued
- Decide guilt or innocence based on arguments
- Explain your reasoning simply
- Be neutral, fair, and concise
"""

# the two agents

defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)

# New agents
plaintiff = LawyerAgent("Plaintiff", PLAINTIFF_SYSTEM)
witness = LawyerAgent("Witness", WITNESS_SYSTEM)
defendant = LawyerAgent("Defendant", DEFENDANT_SYSTEM)
judge = LawyerAgent("Judge", JUDGE_SYSTEM)

class TrialManager:
    def __init__(self, agents: dict, case_background: str):
        self.agents = agents
        self.case = case_background
        self.transcript = []

    def record(self, speaker, message):
        self.transcript.append(f"{speaker.upper()}: {message}\n")

    def run_opening_statements(self):
        print("==== Opening Statements ====\n")
        prosecution_msg = self.agents["prosecution"].respond(f"Opening statement to the Court. Background: {self.case}")
        defense_msg = self.agents["defense"].respond("Opening statement to the Court responding to the prosecution.")
        print("PROSECUTION:", prosecution_msg, "\n")
        print("DEFENSE   :", defense_msg, "\n")
        self.record("prosecution", prosecution_msg)
        self.record("defense", defense_msg)

    def run_interrogation(self):
        print("==== Interrogation Phase ====\n")
        prose_q = self.agents["prosecution"].respond("Interrogate the witness about what they saw.")
        witness_a1 = self.agents["witness"].respond(prose_q)
        prose_follow = self.agents["prosecution"].respond(f"The witness said: {witness_a1}. Respond or continue questioning.")

        defense_q = self.agents["defense"].respond("Cross-examine the witness based on their testimony.")
        witness_a2 = self.agents["witness"].respond(defense_q)
        defense_follow = self.agents["defense"].respond(f"The witness said: {witness_a2}. Respond or continue questioning.")

        for role, msg in [("prosecution", prose_q), ("witness", witness_a1),
                          ("prosecution", prose_follow), ("defense", defense_q),
                          ("witness", witness_a2), ("defense", defense_follow)]:
            self.record(role, msg)

    def run_closing_statements(self):
        print("==== Closing Statements ====\n")
        prose = self.agents["prosecution"].respond("Give your closing statement.")
        defense = self.agents["defense"].respond("Give your closing statement.")
        print("PROSECUTION:", prose, "\n")
        print("DEFENSE   :", defense, "\n")
        self.record("prosecution", prose)
        self.record("defense", defense)

    def judge_verdict(self):
        print("==== Judge's Verdict ====\n")
        max_chars = 1000  # tweak this if needed
        summary = "\n".join(self.transcript)[-max_chars:]
        summary = summary.strip().split("####")[0].strip()


        verdict = self.agents["judge"].respond(
            f"You are the judge. Here's the trial transcript:\n{summary}\nNow give your verdict and explain your reasoning.",
            max_new_tokens=300
        )

        print("JUDGE:", verdict)
        self.record("judge", verdict)



# ==== RUN THE SIMULATION ====

case_background = (
    "The State alleges that John Doe stole proprietary algorithms from his former employer "
    "and used them at a competitor. The charge is felony theft of trade secrets. "
    "No physical evidence shows direct copying, but server logs indicate large downloads "
    "two days before Doe resigned."
)

agents = {
    "prosecution": prosecution,
    "defense": defense,
    "plaintiff": plaintiff,
    "witness": witness,
    "defendant": defendant,
    "judge": judge,
}

trial = TrialManager(agents, case_background)

trial.run_opening_statements()
trial.run_interrogation()
trial.run_closing_statements()
trial.judge_verdict()


==== Opening Statements ====

PROSECUTION: Ladies and gentlemen of the jury, my name is Jordan Blake, and I represent the State in the case against Mr. John Doe, who stands accused of a serious crime: theft of trade secrets. This charge is not taken lightly, as it undermines the very foundation of fair competition and innovation that our economy depends on.


Mr. Doe worked at XYZ Corporation, a company known for its pioneering work in the field of artificial intelligence. XYZ Corporation holds proprietary algorithms that are key to their success. Unfortunately, Mr. Doe, after leaving his position, moved to work for a direct competitor, ABC Inc., where he allegedly utilized the same algorithms without permission.


We have presented clear evidence, including server logs from XYZ Corporation, which show unusually large data transfers from the company’s server to an unrecognized server just two days before Mr. Doe’s resignation. While it may not be direct evidence of theft, when coupled 

In [ ]:
# ... (keep all your existing imports and LawyerAgent class)

class CourtAgent(LawyerAgent):
    def __init__(self, name: str, role: str, case_text: str):
        system_prompt = f"""
        **ROLE**: {role} in this case
        **CASE TEXT**: "{case_text[:300]}..."
        **RULES**:
        1. Respond ONLY about this specific case
        2. Never mention other cases/laws
        3. Keep responses under 100 words
        4. If unsure, say "Objection - beyond case scope"
        """
        super().__init__(name, system_prompt)
        self.case_text = case_text

# Load cases
import pandas as pd
from typing import List, Dict

# Load the CSV file without headers
try:
    # Assuming the first column contains the case text
    cases_df = pd.read_csv("data.csv", header=None, names=['full_case_text'])
    print(f"Successfully loaded {len(cases_df)} cases")
except Exception as e:
    print(f"Error loading data: {e}")
    cases_df = pd.DataFrame(columns=['full_case_text'])  # empty dataframe as fallback

# Process cases
results = []
MAX_CASES = min(10, len(cases_df))  # Handle case if fewer than 10 cases exist

for i, row in cases_df.head(MAX_CASES).iterrows():
    try:
        case_text = row['full_case_text']

        # Initialize agents (using your existing LawyerAgent class)
        prosecutor = LawyerAgent("Prosecutor",
                               f"You are prosecuting this case. Case details: {case_text[:300]}...")
        defense = LawyerAgent("Defense",
                             f"You are defending this case. Case details: {case_text[:300]}...")

        # Generate arguments
        prosecution_stmt = prosecutor.respond("Present your opening arguments to the court")
        defense_stmt = defense.respond("Present your response to the prosecution's arguments")

        results.append({
            'case_id': i,
            'case_preview': str(case_text)[:100] + "...",  # str() in case it's not string
            'prosecution': prosecution_stmt,
            'defense': defense_stmt
        })
    except Exception as e:
        print(f"Error processing case {i}: {e}")

# Preview results
print("\nSample outputs:")
for r in results[:2]:
    print(f"\nCase {r['case_id']}: {r['case_preview']}")
    print(f"Prosecution: {r['prosecution'][:100]}...")
    print(f"Defense: {r['defense'][:100]}...")


Successfully loaded 501 cases

Sample outputs:

Case 0: text...
Prosecution: Ladies and Gentlemen of the jury, before you today we have an issue of substantial importance that t...
Defense: I'm sorry, but I cannot generate a response without knowing the specifics of the case at hand. Could...

Case 1:  F. NARIMAN, J. Leave granted. In 2008, the Punjab State Water Supply Sewerage Board, Bhatinda issue...
Prosecution: Your Honor, members of the court, we are here today regarding the dispute between the plaintiff and ...
Defense: In the case at hand, the prosecution has raised several arguments against the extension and augmenta...


In [ ]:
# Convert to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv('court_results.csv', index=False)
files.download('court_results.csv')  # Auto-download in Colab

print(f"\n🎉 Saved {len(results_df)} case results to court_results.csv")
print("Columns:", results_df.columns.tolist())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 Saved 10 case results to court_results.csv
Columns: ['case_id', 'case_preview', 'prosecution', 'defense']
